In [ ]:
import glob
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
import random
from typing import Union, List, Tuple
import matplotlib.pyplot as plt
import datetime
import os
import time

In [ ]:
class CustomDataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, 
                data_directory_path: str = "./",
                labels_directory_path: str = "./",
                is_directory_recursive: bool = False,
                file_type: str = "*csv",
                data_format: str = "2D",
                batch_size: int = 32,
                file_count = None,
                labels_list: Union[List,str] = "cotAlpha",
                to_standardize: bool = False,
                input_shape: Tuple = (13,21),
                include_y_local:bool = False,
                **kwargs,
                ):
        """
        Data Generator to streamline data input to the network direct from the directory.
        Args:
        data_directory_path:
        labels_directory_path: 
        is_directory_recursive: 
        file_type: Default: "*.csv"
                   Adapt the data loader according to file type. For now, it only supports .csv file format.
        data_format: Default: 2D
                     Used to refer to the relevant "recon" files, 2D for 2D pixel array, 3D for time series input,
        batch_size: Default: 32
                    The no. of data points to be included in a single batch.
        file_count: Default: None
                    To limit the no. of .csv files to be used for training.
                    If set to None, all files will be considered as legitimate inputs.
        labels_list: Default: "cotAlpha"
                     Input column name or list of column names to be used as label input to the neural network.
        to_standardize: If set to True, it ensures that batches are normalized prior to being used as inputs
                        for training.
                        Default: False
        input_shape: Default: (13,21) for image input to a 2D feedforward neural network.
                    To reshape the input array per the requirements of the network training.
        """
        self.recon_files = glob.glob(data_directory_path + "recon" + data_format + file_type, 
                                    recursive=is_directory_recursive)
        
        self.recon_files.sort()
        print(len(self.recon_files))
        self.label_files = glob.glob(labels_directory_path + "labels" + file_type, 
                                     recursive=is_directory_recursive)
        self.label_files.sort()
        
        if file_count != None:
            self.recon_files = self.recon_files[:file_count]
            self.label_files = self.label_files[:file_count]
            
        self.batch_size = batch_size
        self.labels_list = labels_list
        self.input_shape = input_shape
        self.to_standardize = to_standardize
        self.include_y_local = include_y_local
        self.on_epoch_end()
        
    
    def standardize(self, x):
        """Applies the normalization configuration in-place to a batch of
        inputs.
        `x` is changed in-place since the function is mainly used internally
        to standardize images and feed them to your network.
        Args:
            x: Batch of inputs to be normalized.
        Returns:
            The inputs, normalized. 
        """
        x -= np.mean(x, keepdims=True)   
        x /= np.std(x, keepdims=True) + 1e-10
        return x
        
            
    def on_epoch_end(self):
        """
        Inherited from the parent class.
        Used to reset indices but not of significance in this datagenerator.
        """
        pass
            
        
    def __getitem__(self, index):
        """
        Used to fetch a batch of inputs (X,y) for the network's training.
        """
        
        file_index = random.randrange(0,len(self.recon_files))
        recon_df = pd.read_csv(self.recon_files[file_index])
        labels_df = pd.read_csv(self.label_files[file_index])
        
        chosen_idxs = random.sample([i for i in range(0,len(labels_df))], self.batch_size)
        
        X = recon_df.iloc[chosen_idxs].values
        if len(self.input_shape) == 1:
            X = X.reshape(self.batch_size,self.input_shape[0])
        if len(self.input_shape) == 2:
            X = X.reshape(self.batch_size,self.input_shape[0],self.input_shape[1])
        elif len(self.input_shape) == 3:
            X = X.reshape(self.batch_size,self.input_shape[0],self.input_shape[1],
                          self.input_shape[2])
        elif len(self.input_shape) == 4:
            X = X.reshape(self.batch_size, self.input_shape[0], self.input_shape[1], 
                          self.input_shape[2], self.input_shape[3])
            
        y = labels_df.iloc[chosen_idxs][self.labels_list].values
        if self.to_standardize:
            X = self.standardize(X)
        if self.include_y_local:
            y_local = labels_df.iloc[chosen_idxs]["y-local"].values
            return [X, y_local], y
        else:
            return X, y
    
    def __len__(self):
        count = 0
        for f in self.label_files:
            x = pd.read_csv(f)
            count += x.shape[0]
        return count // self.batch_size

In [ ]:
# data_directory_path = "./data/raw/"
# file_type = "*.csv"
# data_format = "2D"
# batch_size = 1000
# labels_list = ["cotAlpha", "cotBeta"]
# image_shape = (13,21)
# recon_files = glob.glob(data_directory_path + "recon" + data_format + file_type, recursive=True)
# recon_files.sort()
# label_files = glob.glob(data_directory_path + "labels" + file_type, recursive=True)
# label_files.sort()

# # part 
# # chose a file's index at random and load the csvs corresponding to that index
# index = random.randrange(0,len(recon_files))
# recon_df = pd.read_csv(recon_files[index])
# labels_df = pd.read_csv(label_files[index])
# chosen_idxs = random.sample([i for i in range(0,len(labels_df))], batch_size)
# X = recon_df.iloc[chosen_idxs].values.reshape(batch_size,image_shape[0], image_shape[1])
# y = labels_df.iloc[chosen_idxs][labels_list].values
# X.shape

In [ ]:
from keras.models import Model
from keras.layers import (BatchNormalization, Flatten, Input, Reshape, Dense, MaxPool2D,
                          Conv1D, MaxPool1D, Conv2D, Dropout, Conv3D, concatenate)

class RegModelCotBeta3D:
    def build_image_branch(self,inputs):
#     def build_image_branch(self,inputs,y_local):
        x = Conv2D(64, (2, 2), kernel_initializer = "he_normal",
                   strides=(2, 2), activation='relu',
                   data_format = "channels_first")(inputs)
        x = BatchNormalization()(x)
        x = Conv2D(32, (2, 2), kernel_initializer = "he_normal",
                   strides=(2, 2), activation='relu',
                   data_format = "channels_first")(x)
        x = BatchNormalization()(x)
        x = Dropout(rate = 0.15)(x)
        x = Flatten()(x)
#         x = concatenate([x,y_local])
        x = Dense(32, kernel_initializer = "he_normal", activation='relu', use_bias = False)(x)
        x = BatchNormalization()(x)
        x = Dense(16, kernel_initializer = "he_normal", activation='relu', use_bias = False)(x)
        x = BatchNormalization()(x)
        x = Dense(1, activation= "linear", name = "final_output")(x)

        return x
 
    def assemble_model(self):
        inputs = Input ( shape = (10,13,21) )
        y_local = Input(shape = (1,))
#         outputs = self.build_image_branch(inputs,y_local)
        outputs = self.build_image_branch(inputs)
        model = Model(inputs =[inputs], outputs=outputs, name = "cotBeta_3D_model")
#         model = Model(inputs =[inputs,y_local], outputs=outputs, name = "cotBeta_3D_model")
        print( model.summary() )
        return model

In [ ]:
def custom_loss(y, p):
    
    maxval = 1e6
    minval = 1e-9
    pi = 3.14159265359
    
    mu = tf.reduce_mean(p, axis=1)
    sigma = tf.clip_by_value(tf.reduce_mean(p, axis=1),minval,maxval)
        
    term = tf.clip_by_value(1.0/tf.math.sqrt(2*pi)/sigma*tf.math.exp(-1*(y-mu)*(y-mu)/(2*sigma*sigma)),minval,maxval)
    NLL = tf.clip_by_value(-1*tf.math.log(term),minval,maxval)
    
    return tf.keras.backend.sum(NLL,axis=1)

In [ ]:
start_date = datetime.datetime.now ( )
epochs = 5
batch_size = 7500
val_batch_size = 7500
use_file_size = None
training_generator = CustomDataGenerator(
                                        data_directory_path = "./data/processed/3D-10TS/",
                                        labels_directory_path = "./data/raw/labels/",
                                        is_directory_recursive = False,
                                        file_type = "*csv",
                                        data_format = "3D",
                                        batch_size = batch_size,
                                        file_count = use_file_size,
                                        to_standardize= True,
                                        include_y_local= False,
                                        labels_list = "cotBeta",
                                        input_shape = (10,13,21))

validation_generator = CustomDataGenerator(
                                        data_directory_path = "./data/processed/3D-10TS/",
                                        labels_directory_path = "./data/raw/labels/",
                                        is_directory_recursive = False,
                                        file_type = "*csv",
                                        data_format = "3D",
                                        batch_size = val_batch_size,
                                        file_count = 25,
                                        to_standardize= True,
                                        include_y_local= False,
                                        labels_list = "cotBeta",
                                        input_shape = (10,13,21))

model = RegModelCotBeta3D().assemble_model()

model.compile(optimizer = tf.keras.optimizers.Nadam(lr = 2.75e-3), 
              loss = custom_loss)

history = model.fit(x=training_generator,
                    validation_data=validation_generator,
                    epochs=epochs,
                    verbose = True)

end_date = datetime.datetime.now ( )
print ( "Total Training Time = {}".format (end_date - start_date ) )

In [ ]:
plt.plot(history.history["loss"], color = "blue",marker = "x")
plt.plot(history.history["val_loss"], color ="red", marker = ".")
# plt.xticks([0,1,2,3,4,5,6,7,8,9,10])
plt.show()

### Test Phase:

In [ ]:
test_recon = pd.read_csv("./data/test/recon3D10_test.csv")
test_labels_df = pd.read_csv("./data/test/labels_test.csv")
test_labels = test_labels_df["cotBeta"]
test_y_local = test_labels_df["y-local"]
print(test_recon.shape, test_labels.shape)

In [ ]:
test_inputs = test_recon.values.reshape(test_recon.shape[0],10,13,21)

def standardize(x):
    """Applies the normalization configuration in-place to a batch of
    inputs.
    `x` is changed in-place since the function is mainly used internally
    to standardize images and feed them to your network.
    Args:
        x: Batch of inputs to be normalized.
    Returns:
        The inputs, normalized. 
    """
    x -= np.mean(x, keepdims=True)   
    x /= np.std(x, keepdims=True) + 1e-10
    return x

test_inputs = standardize(test_inputs)

cotBeta_predictions = model.predict(test_inputs, batch_size = 1000)
# cotBeta_predictions = model.predict([test_inputs,test_y_local.values], batch_size = 1000)
df = pd.DataFrame(cotBeta_predictions[:,0], columns = ["preds"])
df["truth"] = test_labels

In [ ]:
plt.figure(figsize = (7,5), dpi = 80)
plt.hist(df["truth"], bins = 100, color = "darkblue", alpha = 0.5, )
plt.hist(df["preds"], bins = 100, color = "red", alpha = 0.3)
# plt.xlim([-10,10])
# plt.xticks([i for i in range(10, -11, -2)])
plt.ylabel("frequency")
plt.xlabel("cotBeta")
plt.legend(["truth", "preds"])
plt.grid(which='major', linestyle='-', linewidth='0.5', alpha=0.5,color='black')
plt.show()

In [ ]:
# df.to_csv("./results.csv", index = False)

In [ ]:
def compute_profile(x, y, nbin=100):
    
    # use of the 2d hist by numpy to avoid plotting
    h, xe, ye = np.histogram2d(x,y,nbin)
    
    # bin width
    xbinw = xe[1]-xe[0]

    # getting the mean and RMS values of each vertical slice of the 2D distribution
    # also the x valuse should be recomputed because of the possibility of empty slices
    x_array      = []
    x_slice_mean = []
    x_slice_rms  = []
    for i in range(xe.size-1):
        yvals = y[ (x>xe[i]) & (x<=xe[i+1]) ]
        if yvals.size>0: # do not fill the quanties for empty slices
            x_array.append(xe[i]+ xbinw/2)
            x_slice_mean.append( yvals.mean())
            x_slice_rms.append( yvals.std())
    x_array = np.array(x_array)
    x_slice_mean = np.array(x_slice_mean)
    x_slice_rms = np.array(x_slice_rms)

    return x_array, x_slice_mean, x_slice_rms

plt.figure(figsize = (7,4), dpi= 90)
p_x, p_mean, p_rms = compute_profile(x = df["truth"],
                                     y = df["truth"] - df["preds"],
                                     nbin = 50)
plt.errorbar(p_x, p_mean, p_rms,fmt='_', ecolor='blue', color='blue')


plt.xlabel("Truth")
plt.ylabel("cotBeta Residuals")
# plt.ylim([-0.5,0.5])
# plt.xlim([-0.6,0.6])
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5', alpha=0.5,color='black')
plt.show()

In [ ]:
residuals = df["truth"] - df["preds"]
print(np.mean(residuals), np.std(residuals))

In [ ]:
# model.save("./models/keras_cnn_cotBeta_3d_20TS.h5")
# model = keras.models.load_model("./models/keras_cnn_cotBeta_3d_05TS.h5")